In [33]:
FILENAME = "taboo_cards_mod.txt"

def read_in(filename):
    """
    Reads in transcribed Taboo card words contained in the given file and returns them in an enumerated list.
    """
    file_lines = []
    
    with open(filename, "r") as myfile:
        
        # Go through every line, saving non-empty ones to the list file_lines.
        for line in myfile:
            if line.strip() != '':             
                file_lines.append(line.strip()) 
                
    return list(enumerate(file_lines))
    
            
def format_cards(enum_list):
    """
    Given an enumerated list (output of read_in()), formats the contents as a dictionary (key = MW, values = list of TWs)
    """
    
    # Initialise dictionary to contain card data.
    card_dict = dict()
    
    # Assign MWs (every sixth word in the enumerated list) as dictionary keys, and create a list for the dict's
    # value consisting of the five following words (the TWs); the word[1:] removes the dash from the beginning of
    # each TW's string.

    for enum, wd in enum_list:
        if enum % 6 == 0:
            card_dict[wd] = [word[1:] for num, word in enum_list[enum+1:enum+6]]
        
    return card_dict
        

enum_lines = read_in(FILENAME)
cards = format_cards(enum_lines)